In [80]:
import requests
import xml.etree.ElementTree as ET
import json
import sys
sys.path.append('/python/')
import connections as con

In [2]:
class Geometadata:
    """
    Henter metadata om tabeller fra geodata-info.dk
    """
    def __init__(self):
        self.fileIdentifier_path = "gmd:fileIdentifier/gco:CharacterString"
        self.title_path = "gmd:identificationInfo/gmd:MD_DataIdentification/gmd:citation/gmd:CI_Citation/gmd:title/gco:CharacterString"
        self.abstract_path = "gmd:identificationInfo/gmd:MD_DataIdentification/gmd:abstract/gco:CharacterString"
        self.organisationName_path = "gmd:contact/gmd:CI_ResponsibleParty/gmd:organisationName/gco:CharacterString"
    
    def get_xml(self,  gid):
        url = f"https://geodata-info.dk/srv/api/records/{gid}/formatters/xml"
        r = requests.request("GET", url)
        root = ET.fromstring(r.text)
        
        return root
    
    def get_property(self, xml, xml_path):
        
        namespaces = {
            'gmd': 'http://www.isotc211.org/2005/gmd',
            'gco': 'http://www.isotc211.org/2005/gco'
        }
               
        prop = xml.find(xml_path, namespaces)
        
        return prop.text
        
    def get_meta_data(self, gid, properties=['fileIdentifier', 'title', 'abstract', 'organisationName']):
    
        """
        Henter metadata udfra metadata id fra geodata-info.dk
        fileIdentifier
        title
        abstract
        organisationName  
        """
        
        root = self.get_xml(gid)
        
        props = {}

        for prop in properties:
            prop = str(prop).lower()
            
            if prop == "fileidentifier":
                props.update({"fileIdentifier" : self.get_property(root, self.fileIdentifier_path)})
            elif prop == "title":
                props.update({"title" : self.get_property(root, self.title_path)})
            elif prop == "abstract":
                props.update({"abstract" : self.get_property(root, self.abstract_path)})
            elif prop == "organisationname":
                props.update({"organisationName" : self.get_property(root, self.organisationName_path)})
                
        return props

# Berig tabeller med metadata fra Geodata-info

In [3]:
import pandas as pd

In [4]:
meta = Geometadata()

In [5]:
query = """
    select "_key_", RIGHT(value, length(value) - 5) id
    from SETTINGS.GEOMETRY_COLUMNS_JOIN, jsonb_array_elements_text(tags)
    where value like '_gdi:%%'
"""

In [6]:
df, engine = con.sql_to_dataframe('production', query)

In [7]:
properties=['title', 'abstract', 'organisationName']

In [8]:
df[properties] = df['id'].apply(lambda x: pd.Series(meta.get_meta_data(x, properties).values()))

In [9]:
df['title'] = df.title.apply(lambda x: list(x)[0]) 
df['abstract'] = df.abstract.apply(lambda x: list(x)[1]) 
df['organisationName'] = df.organisationName.apply(lambda x: list(x)[2]) 

In [10]:
df

,_key_,id,title,abstract,organisationName
0,_01_fysisk_plan_og_naturbeskyt._01_05_bes_natu...,fb9e06dc-9f02-42ac-bf1c-e0d662b81f4f,Beskyttede naturtyper,"Registrering af naturtyper, som er beskyttet e...",Danmarks Miljøportal
1,_00_grundkort._00_02_fredskov.the_geom,3862f6c0-797b-46db-94eb-8baf2d0be5e1,Fredskov,"Arealer, der er noteret med fredskovpligt i ma...",Miljøstyrelsen
2,_01_fysisk_plan_og_naturbeskyt._01_05_kirkebyg...,95dbc473-2269-4f29-8c5e-64620ae15bea,Kirkebyggelinjer,Kirkebyggelinien er en 300 m beskyttelseszone ...,Danmarks Miljøportal
3,_01_fysisk_plan_og_naturbeskyt.bes_naturtyper....,fb9e06dc-9f02-42ac-bf1c-e0d662b81f4f,Beskyttede naturtyper,"Registrering af naturtyper, som er beskyttet e...",Danmarks Miljøportal


# To PostgreSQL

In [ ]:
df.to_sql('geodatainfo_meta')

# Gammelt

In [20]:
def add_metadata(gid, properties, df):
    """
    Add metadata columns to dataframe
    """
    data = meta.get_meta_data(gid, properties)
    print(data)
   
    for key in data:
        #print(df[key], data[key])
        df[key] = data[key]

In [2]:
url = "https://geodata-info.dk/srv/api/records/fb9e06dc-9f02-42ac-bf1c-e0d662b81f4f/formatters/xml"
r = requests.request("GET", url)

In [3]:
root = ET.fromstring(r.text)

<Response [200]>

In [4]:
namespaces = {
    'gmd': 'http://www.isotc211.org/2005/gmd',
    'gco': 'http://www.isotc211.org/2005/gco'
}

In [16]:
base = "gmd:identificationInfo/gmd:MD_DataIdentification/"
abs_path = f"{base}gmd:abstract/gco:CharacterString"

In [17]:
abs_path

'gmd:identificationInfo/gmd:MD_DataIdentification/gmd:abstract/gco:CharacterString'

In [6]:
abstact = root.find(path, namespaces)

In [7]:
abstact.text

'Registrering af naturtyper, som er beskyttet efter naturbeskyttelseslovens § 3.\n\nDisse er:\n·          søer og vandhuller med et areal på mindst 100 m2, \n·          moser, enge, strandenge, strandsumpe, heder og overdrev med et areal på mindst 2500 m2,\n·          "mosaikker" af ovennævnte naturtyper med et areal på mindst 2500 m2,\n·          visse udpegede vandløb  & \n·          alle moser i f. m. beskyttede vandhuller, søer eller vandløb.\nBeskyttelsen gælder uanset om der er en eller flere ejere. Tilstandsændring af beskyttet natur kræver tilladelse (dispensation).\nEt beskyttet areal kan i årenes løb ændre sig så meget, at det ikke længere er beskyttet. Omvendt kan et areal som i dag ikke er beskyttet, ændre sig, så det bliver det.\nDet er de faktiske forhold på arealet (størrelse, botanik, omlægningshyppighed m. m.) der afgør, om det er beskyttet eller ej. Registreringen er derfor vejledende\nHvorvidt et areal er beskyttet eller ej, vil i tvivlstilfælde blive afgjort ved en 

# HENT DATA WFS

Tabelnavn (typename) fra WFS'er GC2 scheduler jobs udtrækkes

In [65]:
wfs = pd.read_csv('data/wfs.csv')

In [74]:
def typename(url):
    for item in url.split('&'):
        if item.split('=')[0].lower() == 'typename':
            return (item.split('=')[1])

In [77]:
wfs['typename'] = wfs['url'].apply(lambda x: typename(x))

In [79]:
wfs.head()

,url,name,schema,typename
0,http://geoserver.surfacewater.miljoeportal.dk/...,smaadyrsfauna_vandloeb,_06_spildevand_vandloeb,DMP:OVF_SmaadyrsFauna_Vandlob
1,http://geoservice.plansystem.dk/wfs?version=1....,_01_02_naturbeskyttelsesomraade_vedtaget,public,pdk:theme_pdk_naturbeskyttelsesomraade_vedtaget_v
2,https://geoserver.plandata.dk/geoserver/wfs?ve...,_01_02_tilslutningspligtomraade_vedtage,_01_fysisk_plan_og_naturbeskyt,pdk:theme_pdk_tilslutningspligtomraade_vedtaget_v
3,https://geoserver.plandata.dk/geoserver/wfs?SE...,varmeforsyningsomraader_vedtagne,_13_forsyning,pdk:theme_pdk_forsyningomraade_vedtaget_v
4,https://geoserver.plandata.dk/geoserver/wfs?ve...,_01_02_detailhandel_vedtaget,_01_fysisk_plan_og_naturbeskyt,pdk:theme_pdk_detailhandel_vedtaget_v


## FIND uuid for hver typename

In [98]:
def get_uuid(typename):
    url = "https://www.geodata-info.dk/srv/dan/q"

    querystring = {
        "_content_type":"json",
        "any":typename,
        "bucket":"s101",
        "facet.q":"",
        "fast":"index",
        "from":"1",
        "resultType":"details",
        "sortBy":"relevance",
        "to":"20"
    }

    headers = {
        'Cache-Control': "no-cache"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    j = response.json()
    
    try:
        return j['metadata']['geonet:info']['uuid']
    except Exception as e:
        print(e)

In [109]:
get_uuid('dmp:AA_BES_LINJER')

'metadata'


In [100]:
wfs['uuid'] = wfs.typename.apply(get_uuid)

'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
list indices must be integers or slices, not str
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
list indices must be integers or slices, not str
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
list indices must be integers or slices, not str
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
'metadata'
list indices must be integers or slices, not str
list indices must be integers or slices, not str
'metadata'
'metada

In [107]:
wfs[wfs.uuid != 'None']

,url,name,schema,typename,uuid
0,http://geoserver.surfacewater.miljoeportal.dk/...,smaadyrsfauna_vandloeb,_06_spildevand_vandloeb,DMP:OVF_SmaadyrsFauna_Vandlob,None
1,http://geoservice.plansystem.dk/wfs?version=1....,_01_02_naturbeskyttelsesomraade_vedtaget,public,pdk:theme_pdk_naturbeskyttelsesomraade_vedtaget_v,None
2,https://geoserver.plandata.dk/geoserver/wfs?ve...,_01_02_tilslutningspligtomraade_vedtage,_01_fysisk_plan_og_naturbeskyt,pdk:theme_pdk_tilslutningspligtomraade_vedtaget_v,None
3,https://geoserver.plandata.dk/geoserver/wfs?SE...,varmeforsyningsomraader_vedtagne,_13_forsyning,pdk:theme_pdk_forsyningomraade_vedtaget_v,None
4,https://geoserver.plandata.dk/geoserver/wfs?ve...,_01_02_detailhandel_vedtaget,_01_fysisk_plan_og_naturbeskyt,pdk:theme_pdk_detailhandel_vedtaget_v,None
5,https://vmgeoserver.vd.dk/geoserver/VD/wfs?SER...,koerebanebredde,_05_veje_trafik,VD:vm_koerebanebredde,None
6,https://geocloud.vd.dk/ANL/wfs?SERVICE=WFS&VER...,vejdirektoratets_anlaegsprojekter2,_05_veje_trafik,ANL:Vejprojekter_oversigt,None
7,_system.wfs_grid|http://kortforsyningen.kms.dk...,sogn,_00_grundkort,SOGN10,None
8,https://geoserver.plandata.dk/geoserver/wfs?se...,lokalplan_vindmoelle_vedtaget,_01_fysisk_plan_og_naturbeskyt,pdk:theme_pdk_lokalplan_vedtaget_vindmoelle_v,None
9,https://geoserver.plandata.dk/geoserver/wfs?se...,konsekvensomraade_aflyst,_01_fysisk_plan_og_naturbeskyt,pdk:theme_pdk_konsekvensomraade_aflyst_v,None


In [110]:
def get_uuid(typename):
    url = "https://www.geodata-info.dk/srv/dan/q"

    querystring = {
        "_content_type":"json",
        "any":typename,
        "bucket":"s101",
        "facet.q":"",
        "fast":"index",
        "from":"1",
        "resultType":"details",
        "sortBy":"relevance",
        "to":"20"
    }

    headers = {
        'Cache-Control': "no-cache"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    j = response.json()
    
    return j

In [111]:
get_uuid('dmp:AA_BES_LINJER')

{'@from': '1',
 '@selected': '0',
 '@to': '0',
 'summary': {'@count': '0',
  '@type': 'local',
  'dimension': [{'@label': 'types', '@name': 'type'},
   {'@label': 'topicCats', '@name': 'topicCat'},
   {'@label': 'inspireThemesURI', '@name': 'inspireThemeURI'},
   {'@label': 'inspireThemeCluster', '@name': 'inspireThemeCluster'},
   {'@label': 'keywords', '@name': 'keyword'},
   {'@label': 'orgNames', '@name': 'orgName'},
   {'@label': 'sourceCatalog', '@name': 'sourceCatalog'},
   {'@label': 'createDateYears', '@name': 'createDateYear'},
   {'@label': 'formats', '@name': 'format'},
   {'@label': 'spatialRepresentationTypes',
    '@name': 'spatialRepresentationType'},
   {'@label': 'maintenanceAndUpdateFrequencies',
    '@name': 'maintenanceAndUpdateFrequency'},
   {'@label': 'status', '@name': 'status'},
   {'@label': 'serviceTypes', '@name': 'serviceType'},
   {'@label': 'denominators', '@name': 'denominator'},
   {'@label': 'resolutions', '@name': 'resolution'}]}}